# Assay and run directory

In [ ]:
assay_name = "CHEMBL3888429_cleaned"
run_name = "JAK2_smiles_GA"
run_name_low = str.lower(run_name)
rundir = "results/jak2/smiles_ga/CHEMBL3888429_cleaned/"
pca_components = 50
kmeans = True 
runs_to_display = ['range_physchem_range_ecfp4_counts', 'range_ecfp4_counts', 'range_physchem_range_ecfp4',
                   'smiles_validity_ecfp4', 'range_physchem', 
                   'maxsim_ecfp4', 'range_qed', 'range_ecfp4']

# Analysis

In [ ]:
import os
import numpy as np
import json
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.decomposition import PCA
from utils import data_split, ClusterFps, get_n_clusters, ecfp4, find_cluster, max_tanimoto_similarity, indexes_identical_fps, average_tanimoto_similarity, tanimoto_similarities, calculateScore, Descriptors, quantitative_analysis

import matplotlib.pyplot as plt
import math
import scipy
import pandas as pd
import seaborn as sns 
import matplotlib
from sklearn.cluster import KMeans

font = {'family' : 'normal',
        'size'   : 35}

matplotlib.use("pgf")
# Needed for figures
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'font.size': 35,
    'text.usetex': True,
    'pgf.rcfonts': False,
})


matplotlib.rcParams['axes.unicode_minus'] = False
sns.set()
%matplotlib inline 

In [ ]:
# Load classifiers and test set for downstream analysis 
scores_test, smiles_test, activities_test, clf = data_split(assay_name)

In [ ]:
# Get actives and inactives from test set 
actives = []
inactives = []
smiles_actives = []
smiles_inactives = []

for i, s in enumerate(smiles_test):
    mol = Chem.MolFromSmiles(s)
    if float(activities_test[i])==1:
        actives.append(AllChem.GetMorganFingerprintAsBitVect(mol, 2))
        smiles_actives.append(s)
    else:
        inactives.append(AllChem.GetMorganFingerprintAsBitVect(mol, 2))
        smiles_inactives.append(s)


In [ ]:
# Extract runs and results from generative models 
runs = []
smiles_list = []
scores_list = []
scores_trajectories = []
replica_per_run = {}
top_kept_per_run = {}

all_ads = os.listdir(rundir)

try:
    all_ads.remove('ecfp4_filters_validity_ecfp4')
except:
    pass

print(all_ads)

for directory in all_ads:
    smiles = []
    scores = []
    trajectories = []
    top_kept = []
    replica = 0

    
    for result_dir in os.listdir(os.path.join(rundir, directory)):
        with open(os.path.join(rundir, directory, result_dir, 'results.json'), 'r') as f:
            results = json.load(f)
            curr_smiles = [row['smiles'] for row in results['statistics']][-1]
            smiles.extend(curr_smiles)
            top_kept.extend([len(curr_smiles)])
            scores.extend([row['preds']['scores'] for row in results['statistics']][-1])
            trajectories.extend([np.array(row['preds']['scores']) for row in results['statistics']])
            average_trajectories = np.array([np.array(row['preds']['scores']) for row in results['statistics']])
            replica +=1
    if len(smiles)>0:
        curr_run = directory[6:]
        runs.append(curr_run)
        scores_trajectories.append(average_trajectories)
        smiles_list.append(smiles)
        scores_list.append(scores)
        replica_per_run[curr_run] = replica
        top_kept_per_run[curr_run] = top_kept

In [ ]:
from rdkit.Chem.Draw import MolsToGridImage

def show_grid_mols(ad):
    mols = [Chem.MolFromSmiles(s) for s in smiles_list[runs.index(ad)][:50]]
    img = MolsToGridImage(mols, legends=[str(i) for i in range(len(mols))], molsPerRow=5, returnPNG=True)
    return img

def save_mols(ad, indexes):
    mols = []
    for i, s in enumerate(smiles_list[runs.index(ad)][:50]):
        if i in indexes:
            mols.append(Chem.MolFromSmiles(s))
    img=MolsToGridImage(mols, molsPerRow=len(mols), subImgSize=(300, 160), useSVG=True)
    img_pathname = 'images/' + run_name + '/' + run_name_low + '_typical_structures_' + '{}.svg'.format(ad)
    with open(img_pathname , 'w') as f_handle:
        f_handle.write(img.data)
        

In [ ]:
# Summary of the number of replica per run and of the number of molecules generated per replica
for k,v in top_kept_per_run.items():
    print(k, len(v), np.mean(np.array(v)),np.std(np.array(v)))

In [ ]:
# Cluster the test set for downstream analysis 
X = np.array(actives + inactives)
clustering = KMeans(n_clusters=5, random_state=0).fit(X)

kmeans_clusters = clustering.labels_
training_set_clusters = ClusterFps(actives + inactives, cutoff=0.5)
centroids = [cluster[0] for cluster in training_set_clusters]
fp_list = actives + inactives
centroids_fp = [fp_list[centroid] for centroid in centroids]

In [ ]:
# Get all fingerprints to run all analysis faster 
fps_active_as_array = []
fps_active_as_bitv = []
fps_inactive_as_array = []
fps_inactive_as_bitv = []
fps_generated_as_array = []
fps_generated_as_bitv = []

for s in smiles_actives:
    mol = Chem.MolFromSmiles(s)
    fps_active_as_array.append(np.array(AllChem.GetMorganFingerprintAsBitVect(mol, 2)))
    fps_active_as_bitv.append(AllChem.GetMorganFingerprintAsBitVect(mol, 2))
    
for s in smiles_inactives:
    mol = Chem.MolFromSmiles(s)
    fps_inactive_as_array.append(np.array(AllChem.GetMorganFingerprintAsBitVect(mol, 2)))
    fps_inactive_as_bitv.append(AllChem.GetMorganFingerprintAsBitVect(mol, 2))
    
for i, run in enumerate(runs):
    smiles_generated = smiles_list[i]
    curr_fps_generated_as_array = []
    curr_fps_generated_as_bv = []
    for s in smiles_generated:
        mol = Chem.MolFromSmiles(s)
        curr_fps_generated_as_array.append(np.array(AllChem.GetMorganFingerprintAsBitVect(mol, 2)))
        curr_fps_generated_as_bv.append(AllChem.GetMorganFingerprintAsBitVect(mol, 2))
    fps_generated_as_array.append(curr_fps_generated_as_array)
    fps_generated_as_bitv.append(curr_fps_generated_as_bv)
    

In [ ]:
# Fits PCA on test set 
pca = PCA(n_components=2).fit(np.concatenate((fps_inactive_as_array,fps_inactive_as_array)))
pca_active = pca.transform(fps_active_as_array)
pca_inactive = pca.transform(fps_inactive_as_array)

pca_generated = []

for i, run in enumerate(runs):
    curr_fps_generated = fps_generated_as_array[i]
    pca_generated.append(pca.transform(curr_fps_generated))

In [ ]:
xmin_ymin = np.amin(np.vstack([np.amin(p, axis=0) for p in pca_generated]), axis=0).tolist()

In [ ]:
xmax_ymax = np.amax(np.vstack([np.amax(p, axis=0) for p in pca_generated]), axis=0).tolist()

In [ ]:
x_lims = [xmin_ymin[0]] + [xmax_ymax[0]]
y_lims = [xmin_ymin[1]] + [xmax_ymax[1]]

In [ ]:
fig, axs = plt.subplots(figsize = (20, 20), nrows=max(1, math.ceil(len(runs_to_display)/3)) , ncols=3)
fig.tight_layout(pad=1.5)

for i, run in enumerate(runs_to_display):
    index = runs.index(run)
    pca_gen = pca_generated[index]
    ax = axs.flatten()[i]
    ax.set_title(run.replace('_', ' '), fontsize=30)
    ax.set_ylim(y_lims)
    ax.set_xlim(x_lims)
    ax.scatter(pca_gen[:, 0], pca_gen[:, 1], c='b', label = 'generated', alpha=0.25)
    ax.scatter(pca_inactive[:, 0], pca_inactive[:, 1], c='r', label = 'inactives', alpha=0.25)
    ax.scatter(pca_active[:, 0], pca_active[:, 1], c='g', label = 'actives', alpha=0.75)
    ax.tick_params(axis='both', which='major', labelsize=25)
    ax.tick_params(axis='both', which='minor', labelsize=25)
    if i==len(runs_to_display)-1: 
        ax.legend(loc='lower right',
              ncol=1, fancybox=True, shadow=True, fontsize=30)
    
plt.tight_layout()
fig_name = 'images/' + run_name + '/PCA.pdf'
plt.savefig(fig_name, dpi=300)   

In [ ]:
# Computes entropy for the different runs 
pca_generated = []
entropies = []
entropies_all_per_rep = []
clusters = []
clusters_all_per_rep = []

    
pca = PCA(n_components=2).fit(np.concatenate((fps_active_as_array,fps_inactive_as_array)))
pca_active = pca.transform(fps_active_as_array)
pca_inactive = pca.transform(fps_inactive_as_array)
predicted_actives = np.where(clf.predict_proba(ecfp4(smiles_actives))[:, 1]>0.5)[0]

for i, run in enumerate(runs):
    all_curr_fps_generated = fps_generated_as_array[i]
    all_curr_fps_generated_bitv = fps_generated_as_bitv[i]
    entropy_per_rep =[]
    clusters_per_rep = []
    nb_replicas = replica_per_run[run]    
    for replica in range(nb_replicas):
        curr_fps_generated = all_curr_fps_generated[replica*top_kept_per_run[run][replica]:(replica+1)*top_kept_per_run[run][replica]]
        curr_fps_generated_bitv = all_curr_fps_generated_bitv[replica*top_kept_per_run[run][replica]:(replica+1)*top_kept_per_run[run][replica]]
        pca_generated.append(pca.transform(curr_fps_generated))
        if kmeans:
            curr_clusters = clustering.predict(curr_fps_generated)  
        else:
            curr_clusters = [find_cluster(fp, centroids_fp) for fp in curr_fps_generated_bitv]
        probs = []
        for i in range(len(np.unique(centroids))):
            probs.append(len(np.where(np.array(curr_clusters)==i)[0])/len(curr_clusters))
        curr_entropy = scipy.stats.entropy(probs)/ np.log(len(curr_clusters))
        entropy_per_rep.append(curr_entropy)
        clusters_per_rep.append(curr_clusters)
    entropies_all_per_rep.append(entropy_per_rep)
    clusters_all_per_rep.append(clusters_per_rep)
entropies = [x for sublist in entropies_all_per_rep for x in sublist]   
clusters = [x for sublist in clusters_all_per_rep for x in sublist]   

In [ ]:
# compute number of clusters using RDKit's Butina algorithm

n_clusters = []
n_clusters_sd = []

for i, run in enumerate(runs):
    all_curr_fps_generated = fps_generated_as_bitv[i]
    all_curr_n_clusters = []
    nb_replicas = replica_per_run[run]
    for replica in range(nb_replicas):
        curr_fps_generated = all_curr_fps_generated[replica*top_kept_per_run[run][replica]:(replica+1)*top_kept_per_run[run][replica]]
        all_curr_n_clusters.append(get_n_clusters(curr_fps_generated, cutoff=0.5))
    n_clusters.append(np.mean(all_curr_n_clusters))
    n_clusters_sd.append(np.std(all_curr_n_clusters))

n_clusters_display = [ str(round(m,2))+" ("+str(round(sd,2))+")" for m,sd in zip(n_clusters, n_clusters_sd)]

In [ ]:
xmin_ymin = np.amin(np.vstack([np.amin(p, axis=0) for p in pca_generated]), axis=0).tolist()
xmax_ymax = np.amax(np.vstack([np.amax(p, axis=0) for p in pca_generated]), axis=0).tolist()
x_lims = [xmin_ymin[0]] + [xmax_ymax[0]]
y_lims = [xmin_ymin[1]] + [xmax_ymax[1]]

In [ ]:
# Displays proportion of actives recovered at distance < 0.1 
fig = plt.figure(figsize=(25, 25))

novel_01_all_per_rep = []
titles = []

predicted_actives = np.where(clf.predict_proba(ecfp4(smiles_actives))[:, 1]>0.5)[0]
for i, run in enumerate(runs):
    all_curr_fps_generated = fps_generated_as_bitv[i]
    novel_01_per_rep = []
    nb_replicas = replica_per_run[run]
    for replica in range(nb_replicas):
        curr_fps_generated = all_curr_fps_generated[replica*top_kept_per_run[run][replica]:(replica+1)*top_kept_per_run[run][replica]]
        n_new = 0
        for s in np.array(smiles_actives)[predicted_actives]:
            mol = Chem.MolFromSmiles(s)
            if max_tanimoto_similarity([AllChem.GetMorganFingerprintAsBitVect(mol, 2)], curr_fps_generated)>0.9:
                n_new += 1
        novel_01_per_rep.append(n_new/len(predicted_actives))
        titles.append(run.replace('_', ' '))
    novel_01_all_per_rep.append(novel_01_per_rep)

novel_01 = [x for sublist in novel_01_all_per_rep for x in sublist]    

display_df = pd.DataFrame(zip(titles, novel_01), columns =['Applicability domain', 'Mean proportion of generated structures closer than 0.1 (Tanimoto distance, MFP2 FPs) from test set actives'])
chart = sns.catplot(y="Applicability domain", x='Mean proportion of generated structures closer than 0.1 (Tanimoto distance, MFP2 FPs) from test set actives', kind="bar", data=display_df)
for axes in chart.axes.flat:
    _ = axes.set_xticklabels(axes.get_xticklabels(), rotation=90)
plt.title("Actives recovered at distance $<$ 0.1")
plt.show()

In [ ]:
# Displays proportion of actives recovered 

fig = plt.figure(figsize=(25, 25))

novel_recovered_all_per_rep = []
titles = []

predicted_actives = np.where(clf.predict_proba(ecfp4(smiles_actives))[:, 1]>0.5)[0]
for i, run in enumerate(runs):
    all_curr_fps_generated = fps_generated_as_bitv[i]
    novel_recovered_per_rep = []
    nb_replicas = replica_per_run[run]
    for replica in range(nb_replicas):
        curr_fps_generated = all_curr_fps_generated[replica*top_kept_per_run[run][replica]:(replica+1)*top_kept_per_run[run][replica]]
        n_new = 0
        for s in np.array(smiles_actives)[predicted_actives]:
            mol = Chem.MolFromSmiles(s)
            if max_tanimoto_similarity([AllChem.GetMorganFingerprintAsBitVect(mol, 2)], curr_fps_generated)>0.99:
                n_new += 1
        novel_recovered_per_rep.append(n_new/len(predicted_actives))
        titles.append(run.replace('_', ' '))
    novel_recovered_all_per_rep.append(novel_recovered_per_rep)

novel_recovered = [x for sublist in novel_recovered_all_per_rep for x in sublist]    

display_df = pd.DataFrame(zip(titles, novel_recovered), columns =['Applicability domain', 'Generated structures with similarity=1 (same structure, stereoisomer or very close structure) from test set actives'])
chart = sns.catplot(y="Applicability domain", x="Generated structures with similarity=1 (same structure, stereoisomer or very close structure) from test set actives", kind="bar", data=display_df)
for axes in chart.axes.flat:
    _ = axes.set_xticklabels(axes.get_xticklabels(), rotation=90)
plt.title("Actives recovered")
plt.show()

In [ ]:
# Computes distances to training set molecules to display enrichment 
training_set_molecules_predicted_actives = np.where(clf.predict_proba(ecfp4(smiles_actives+smiles_inactives))[:, 1]>0.5)[0]
training_set_molecules_predicted_inactives = np.where(clf.predict_proba(ecfp4(smiles_actives+smiles_inactives))[:, 1]<=0.5)[0]
average_similarities = []
max_similarities_actives = []
max_similarities_inactives = []
similarities_actives = []
similarities_inactives = []
similarities_to_closest_actives = []
similarities_to_closest_inactives = []
similarities_to_closest_predicted_actives = []
similarities_to_closest_predicted_inactives = []

ordered_runs = ['maxsim_ecfp4',
                'range_physchem_range_ecfp4_counts']
labels = [
 'maxsim ecfp4',
 
 'range physchem + range ecfp4 counts',]

for i, run in enumerate(ordered_runs):
    curr_fps_generated = fps_generated_as_bitv[runs.index(run)]
    average_similarities.append(average_tanimoto_similarity(fps_active_as_bitv, curr_fps_generated))
    max_similarities_actives.append(max_tanimoto_similarity(fps_active_as_bitv, curr_fps_generated))
    max_similarities_inactives.append(max_tanimoto_similarity(fps_active_as_bitv, curr_fps_generated))
    similarities_actives.append(tanimoto_similarities(fps_active_as_bitv, curr_fps_generated))
    similarities_inactives.append(tanimoto_similarities(fps_inactive_as_bitv, curr_fps_generated))
    curr_similarities_to_closest_actives = []
    for fp in fps_active_as_bitv:
        curr_similarities_to_closest_actives.append(max_tanimoto_similarity([fp], curr_fps_generated))
    similarities_to_closest_actives.append(curr_similarities_to_closest_actives)
    curr_similarities_to_closest_inactives = []
    for fp in fps_inactive_as_bitv:
        curr_similarities_to_closest_inactives.append(max_tanimoto_similarity([fp], curr_fps_generated))
    similarities_to_closest_inactives.append(curr_similarities_to_closest_inactives)
    curr_similarities_to_closest_predicted_actives = []
    curr_similarities_to_closest_predicted_inactives = []
    for j, fp in enumerate(fps_active_as_bitv + fps_inactive_as_bitv):
        if j in training_set_molecules_predicted_actives:
            curr_similarities_to_closest_predicted_actives.append(max_tanimoto_similarity([fp], curr_fps_generated))
        if j in training_set_molecules_predicted_inactives:
            curr_similarities_to_closest_predicted_inactives.append(max_tanimoto_similarity([fp], curr_fps_generated))
    similarities_to_closest_predicted_actives.append(curr_similarities_to_closest_predicted_inactives)
    similarities_to_closest_predicted_inactives.append(curr_similarities_to_closest_predicted_inactives)

In [ ]:
# Computes distances to training set molecules to display enrichment 
active_neighbours = []
inactive_neighbours = []

for i, run in enumerate(ordered_runs):
    active_sims = similarities_to_closest_actives[i]
    inactive_sims = similarities_to_closest_inactives[i]
        
    active_sims = np.array(active_sims)
    inactive_sims = np.array(inactive_sims)
    
    sim_tresholds = [0.95, 0.9, 0.85, 0.8, 0.75, 0.7, 0.6, 0.5, 0]
    curr_active_neighbours = []
    curr_inactive_neighbours = []
    sim_tresholds.reverse()
    for s in sim_tresholds:
        try:
            curr_active_neighbours.append(len(np.where(active_sims>s)[0]))
        except:
            curr_active_neighbours.append(0)
        try:
            curr_inactive_neighbours.append(len(np.where(inactive_sims>s)[0]))
            
        except:
            curr_inactive_neighbours.append(0)
    active_neighbours.append(curr_active_neighbours)
    inactive_neighbours.append(curr_inactive_neighbours)

In [ ]:

# Compares enrichment on a good and a bad AD 
labels = ['Bad AD: maxsim ecfp4', 'Good AD: range physchem + range ecfp4 counts']
fig, axs = plt.subplots(figsize = (20, 10), nrows=max(1, math.ceil(len(ordered_runs)/2)) , ncols=2, sharey=True)
fig.tight_layout(pad=1.5)
fig.suptitle('Fraction of actives and inactives among the training set molecules close to the generated molecules',fontsize=30, y = 1.05)

for i, run in enumerate(ordered_runs):
    curr_active_neighbours = active_neighbours[i]
    curr_inactive_neighbours = inactive_neighbours[i]
    ax = axs.flatten()[i]
    ax.set_title(labels[i], fontsize=30)
    ax.bar(np.arange(len(sim_tresholds)), np.array(curr_active_neighbours)/len(fps_active_as_bitv), width=0.25, label='fraction actives')
    ax.bar(np.arange(len(sim_tresholds))+ 0.25, np.array(curr_inactive_neighbours)/len(fps_inactive_as_bitv), width=0.25, label='fraction inactives')
    
    ax.set_xticks(np.arange(len(sim_tresholds)))
    ax.set_xticklabels(['$<$0.5'] + sim_tresholds[1:])
    ax.tick_params(axis='both', which='major', labelsize=25)

    ax.set_ylim(0, 1.05)
    if i == 0 :
        ax.legend(loc=0, fontsize=25)
        
plt.tight_layout()


In [ ]:
# Computes distances to training set molecules to display enrichment 
training_set_molecules_predicted_actives = np.where(clf.predict_proba(ecfp4(smiles_actives+smiles_inactives))[:, 1]>0.5)[0]
training_set_molecules_predicted_inactives = np.where(clf.predict_proba(ecfp4(smiles_actives+smiles_inactives))[:, 1]<=0.5)[0]
average_similarities = []
max_similarities_actives = []
max_similarities_inactives = []
similarities_actives = []
similarities_inactives = []
similarities_to_closest_actives = []
similarities_to_closest_inactives = []
similarities_to_closest_predicted_actives = []
similarities_to_closest_predicted_inactives = []

ordered_runs = ['maxsim_ecfp4',
                'range_physchem_range_ecfp4_counts']
labels = [
 'maxsim ecfp4',
 
 'range physchem + range ecfp4 counts',]

for i, run in enumerate(ordered_runs):
    all_curr_fps_generated = fps_generated_as_bitv[runs.index(run)]
    nb_replicas = replica_per_run[run]
    for replica in range(nb_replicas):
        nb_kept = top_kept_per_run[run][replica]
        curr_fps_generated = all_curr_fps_generated[replica*nb_kept:(replica+1)*nb_kept]
        average_similarities.append(average_tanimoto_similarity(fps_active_as_bitv, curr_fps_generated))
        max_similarities_actives.append(max_tanimoto_similarity(fps_active_as_bitv, curr_fps_generated))
        max_similarities_inactives.append(max_tanimoto_similarity(fps_active_as_bitv, curr_fps_generated))
        similarities_actives.append(tanimoto_similarities(fps_active_as_bitv, curr_fps_generated))
        similarities_inactives.append(tanimoto_similarities(fps_inactive_as_bitv, curr_fps_generated))
        curr_similarities_to_closest_actives = []
        for fp in fps_active_as_bitv:
            curr_similarities_to_closest_actives.append(max_tanimoto_similarity([fp], curr_fps_generated))
        similarities_to_closest_actives.append(curr_similarities_to_closest_actives)
        curr_similarities_to_closest_inactives = []
        for fp in fps_inactive_as_bitv:
            curr_similarities_to_closest_inactives.append(max_tanimoto_similarity([fp], curr_fps_generated))
        similarities_to_closest_inactives.append(curr_similarities_to_closest_inactives)
        curr_similarities_to_closest_predicted_actives = []
        curr_similarities_to_closest_predicted_inactives = []
        for j, fp in enumerate(fps_active_as_bitv + fps_inactive_as_bitv):
            if j in training_set_molecules_predicted_actives:
                curr_similarities_to_closest_predicted_actives.append(max_tanimoto_similarity([fp], curr_fps_generated))
            if j in training_set_molecules_predicted_inactives:
                curr_similarities_to_closest_predicted_inactives.append(max_tanimoto_similarity([fp], curr_fps_generated))
        similarities_to_closest_predicted_actives.append(curr_similarities_to_closest_predicted_inactives)
        similarities_to_closest_predicted_inactives.append(curr_similarities_to_closest_predicted_inactives)

In [ ]:
# Computes distances to training set molecules to display enrichment 
active_neighbours = []
inactive_neighbours = []
run_names = []
curr_index = 0

for i, run in enumerate(ordered_runs):
    for replica in range(nb_replicas):
        active_sims = similarities_to_closest_actives[curr_index+replica]
        inactive_sims = similarities_to_closest_inactives[curr_index+replica]
            
        active_sims = np.array(active_sims)
        inactive_sims = np.array(inactive_sims)
        
        sim_thresholds = [0.95, 0.9, 0.85, 0.8, 0.75, 0.7, 0.6, 0.5, 0]
        curr_active_neighbours = []
        curr_inactive_neighbours = []
        sim_thresholds.reverse()
        for s in sim_thresholds:
            try:
                curr_active_neighbours.append(len(np.where(active_sims>s)[0]))
            except:
                curr_active_neighbours.append(0)
            try:
                curr_inactive_neighbours.append(len(np.where(inactive_sims>s)[0]))
                
            except:
                curr_inactive_neighbours.append(0)
        curr_active_neighbours = [i/len(fps_active_as_bitv) for i in curr_active_neighbours ]
        curr_inactive_neighbours = [i/len(fps_inactive_as_bitv) for i in curr_inactive_neighbours ]
        active_neighbours.append(curr_active_neighbours)
        inactive_neighbours.append(curr_inactive_neighbours)
        run_names.append(run)
    curr_index = curr_index + replica_per_run[run]


In [ ]:
sim_thresholds_colnames = ['$<$0.5'] + [ str(i) for i in sim_thresholds[1:]]
fraction_active_col = [ "Active" for r in run_names]
fraction_inactive_col = [ "Inactive" for r in run_names]

#sim_thresholds_inact = [ "i_"+str(i) for i in sim_thresholds]

display_df = pd.concat([pd.concat([pd.DataFrame(run_names,columns=['Applicability domain']),pd.DataFrame(fraction_active_col,columns=['Set']), pd.DataFrame(np.array(active_neighbours),columns=sim_thresholds_colnames)],axis=1,ignore_index=False), pd.concat([pd.DataFrame(run_names,columns=['Applicability domain']),pd.DataFrame(fraction_inactive_col,columns=['Set']),pd.DataFrame(np.array(inactive_neighbours),columns=sim_thresholds_colnames)],axis=1,ignore_index=False)])
display_df = display_df.melt(id_vars=['Applicability domain','Set'])

In [ ]:
display_df.rename(columns={'variable' : 'Similarity (Tanimoto)', 'value' : 'fraction'},inplace=True)

In [ ]:
chart = sns.catplot(data=display_df, x='Similarity (Tanimoto)', y='fraction', hue="Set", kind="bar", col='Applicability domain')
#fig.tight_layout(pad=1.5)
#chart.fig.suptitle('Fraction of actives and inactives among the training set molecules close to the generated molecules',y = 1.05)

fig_name = 'images/' + run_name + '/training_set_analogs_among_generated.pdf'

chart.savefig(fig_name, dpi=300)

In [ ]:
# Compares enrichment on a good and a bad AD 
labels = ['Bad AD: maxsim ecfp4', 'Good AD: range physchem + range ecfp4 counts']
fig, axs = plt.subplots(figsize = (20, 10), nrows=max(1, math.ceil(len(ordered_runs)/2)) , ncols=2, sharey=True)
fig.tight_layout(pad=1.5)
fig.suptitle('Fraction of actives and inactives among the training set molecules close to the generated molecules',fontsize=30, y = 1.05)

for i, run in enumerate(ordered_runs):
    curr_active_neighbours = active_neighbours[i]
    curr_inactive_neighbours = inactive_neighbours[i]
    ax = axs.flatten()[i]
    ax.set_title(labels[i], fontsize=30)
    ax.bar(np.arange(len(sim_tresholds)), np.array(curr_active_neighbours)/len(fps_active_as_bitv), width=0.25, label='fraction actives')
    ax.bar(np.arange(len(sim_tresholds))+ 0.25, np.array(curr_inactive_neighbours)/len(fps_inactive_as_bitv), width=0.25, label='fraction inactives')
    
    ax.set_xticks(np.arange(len(sim_tresholds)))
    ax.set_xticklabels(['$<$0.5'] + sim_tresholds[1:])
    ax.tick_params(axis='both', which='major', labelsize=25)

    ax.set_ylim(0, 1.05)
    if i == 0 :
        ax.legend(loc=0, fontsize=25)
        
plt.tight_layout()
fig_name = 'images/' + run_name + '/enrichment.pdf'
plt.savefig(fig_name, dpi=300)

In [ ]:
# Computes QED and SAS for the test set 
qed_dataset = []
sas_dataset = []
for s in smiles_actives + smiles_inactives:
    mol = Chem.MolFromSmiles(s)
    qed_dataset.append(Descriptors.qed(mol))
    sas_dataset.append(calculateScore(mol))

In [ ]:
ordered_runs = ['smiles_validity_ecfp4', 'maxsim_ecfp4', 'maxsim_ecfp6', 'maxsim_ap', 'range_qed',
                'range_ecfp4', 'range_ecfp4_counts', 'range_physchem', 'range_physchem_range_ap', 'range_physchem_range_ecfp4', 'range_physchem_range_ecfp6',
                'range_physchem_range_ecfp4_counts', 'range_physchem_maxsim_ecfp4', 'range_physchem_maxsim_ecfp6', 
                'range_physchem_maxsim_ap']
                
# Computes QED and SAS for generated sets 
QED = []
SAS = []

SAS_for_results = []
QED_for_results = []
ad = []
for i, run in enumerate(ordered_runs):
    avg_sims = []
    max_sim = []
    smiles_generated = smiles_list[runs.index(run)]
    qed_generated = []
    sas_generated = []
    for s in smiles_generated:
        mol = Chem.MolFromSmiles(s)
        qed_generated.append(Descriptors.qed(mol))
        sas_generated.append(calculateScore(mol))
        #QED.append(Descriptors.qed(mol))
        #SAS.append(calculateScore(mol))
        ad.append(run)
    SAS_for_results.append(sas_generated)
    QED_for_results.append(qed_generated)

In [ ]:
ordered_runs_4_display = list(reversed(ordered_runs))
SAS_for_results_4display = list(reversed(SAS_for_results))

# Plots SAS distributions 
matplotlib.rcParams.update({'font.size': 35})

plt.figure(figsize= (15, 15))

c = 'blue'
box = plt.boxplot(SAS_for_results_4display, labels = [x.replace('validity_ecfp4', 'validity').replace('_', ' ') for x in ordered_runs_4_display], vert=False, patch_artist=True,
            boxprops=dict(facecolor=c, color=c, alpha=0.5),
            capprops=dict(color=c),
            whiskerprops=dict(color='k'),
            flierprops=dict(color=c, markeredgecolor=c),
            medianprops=dict(color='k'),)



plt.axvline(np.percentile(sas_dataset, 100), ls='--', label='Dataset extreme QED values', c='g')
plt.axvline(np.percentile(sas_dataset, 0), ls='--', c='g')
xmin, xmax, ymin, ymax = plt.axis()
plt.fill_between([np.percentile(sas_dataset, 0), np.percentile(sas_dataset, 100)], ymin, ymax, color='g', alpha=0.1, label = 'Valid QED')
plt.xlabel("Synthetic Accessibility Score Distribution", fontsize=30)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)


plt.tight_layout()
fig_name = 'images/' + run_name + '/SAS_distribution.pdf'
plt.savefig(fig_name, dpi=300)

In [ ]:
QED_for_results_4display = list(reversed(QED_for_results))

# Plots QED distributions 
plt.figure(figsize= (15, 15))
c = 'blue'
box = plt.boxplot(QED_for_results_4display, labels = [x.replace('validity_ecfp4', 'validity').replace('_', ' ') for x in ordered_runs_4_display], vert=False, patch_artist=True,
            boxprops=dict(facecolor=c, color=c, alpha=0.5),
            capprops=dict(color=c),
            whiskerprops=dict(color='k'),
            flierprops=dict(color=c, markeredgecolor=c),
            medianprops=dict(color='k'),)

plt.axvline(np.percentile(qed_dataset, 100), ls='--', label='Dataset extreme QED values', c='g')
plt.axvline(np.percentile(qed_dataset, 0), ls='--', c='g')
xmin, xmax, ymin, ymax = plt.axis()
plt.fill_between([np.percentile(qed_dataset, 0), np.percentile(qed_dataset, 100)], ymin, ymax, color='g', alpha=0.1, label = 'Valid QED')
plt.xlabel("QED Distribution", fontsize=30)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)


plt.tight_layout()
fig_name = 'images/' + run_name + '/QED_distribution.pdf'
plt.savefig(fig_name, dpi=300)

# Comparing different AD's outputs

In [ ]:
from utils import qualitative_analysis, return_distribution_cycle_size, return_distribution_mw, return_distribution_halogen, return_distribution_heteroatoms, return_distribution_radicals, return_distribution_sulphur, qualitative_analysis

In [ ]:
# Example on maxsim AP applicability domain of comparison of properties distribution with the test set 
properties = ['cycle_size', 'mw', 'halogen', 'sulphur', 'heteroatoms', 'radicals']
properties_name = ['Cycle size', 'Molecular Weight', 'Halogen atoms', 'Sulfur atoms', 'Heteroatom-heteroatom bonds', 'Unpaired electrons']

fig, axs = plt.subplots(figsize = (20, 20), nrows=max(1, math.ceil(len(properties)/3)) , ncols=3)

fig.tight_layout(pad=1.5)
for i, prop in enumerate(properties):
    ax = axs.flatten()[i]
    ax.set_xlabel(properties_name[i], fontsize=30)
    ref_distribution = locals()["return_distribution_" + prop](smiles_test)
    generated_distribution = locals()["return_distribution_" + prop](smiles_list[0])
    if i%3==0:
        ax.set_ylabel("Frequency", fontsize=30)
    ax.hist([ref_distribution, generated_distribution], label=['Dataset', 'Generated'], density=True)
    if i==len(properties)-1:
        ax.legend(loc='upper right',
              ncol=1, fancybox=True, shadow=True, fontsize=30)
    ax.tick_params(axis='both', which='major', labelsize=25)
    ax.tick_params(axis='both', which='minor', labelsize=25)
fig.tight_layout(pad=3.0)


In [ ]:
best_methods = ['range_physchem_range_ap', 'range_ecfp4_counts', 'range_physchem_range_ecfp4_counts', 'range_physchem_range_ecfp6', 'range_physchem_range_ecfp4']

In [ ]:
# Computes average similarities between runs from the "best methods"
average_similarities = []
max_similarities = []
common_molecules = {}
for i, run in enumerate(best_methods):
    avg_sims = []
    max_sim = []
    smiles_generated = smiles_list[runs.index(run)]
    fps_generated = fps_generated_as_bitv[runs.index(run)]
    for j, comparison_run in enumerate(best_methods):
        smiles_generated = smiles_list[runs.index(comparison_run)]
        fps_other = fps_generated_as_bitv[runs.index(comparison_run)]
        if j<i:
            indexes, indexes_other = indexes_identical_fps(fps_generated, fps_other)
            common_molecules[run + '_vs_' + comparison_run] = [indexes, indexes_other] 
        avg_sims.append(average_tanimoto_similarity(fps_generated, fps_other))
        max_sim.append(max_tanimoto_similarity(fps_generated, fps_other))
    average_similarities.append(avg_sims)
    max_similarities.append(max_sim)

In [ ]:
# Displays average generated set - generated set similarities 
from mpl_toolkits.axes_grid1 import make_axes_locatable

labels = ['range physchem + range ap', 'range ecfp4 counts', 'range physchem + range ecfp4 counts', 'range physchem + range ecfp6', 'range physchem + range ecfp4']
labels_xaxis = ['range physchem \n + range ap', 'range ecfp4 counts', 'range physchem \n + range ecfp4 counts', 'range physchem \n + range ecfp6', 'range physchem \n + range ecfp4']


fig, ax = plt.subplots(figsize = (20, 20), nrows=1 , ncols=1)

fig.tight_layout(pad=1.5)
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)



ax.set_title("Average similarities between generated sets", fontsize=35)

im = ax.imshow(np.array(average_similarities), cmap='Blues')
ax.set_yticks(ticks=range(len(best_methods)))
ax.set_xticks(ticks=range(len(best_methods)))
ax.set_xticklabels(labels_xaxis, rotation=45)
ax.set_yticklabels(labels_xaxis, rotation=45)
ax.tick_params(axis='both', which='major', labelsize=35)
ax.tick_params(axis='both', which='minor', labelsize=35)
#plt.colorbar()
fig.tight_layout(pad=3.0)
cbar = plt.colorbar(im, cax=cax, orientation='vertical')
ax.grid(False)
cbar.ax.tick_params(labelsize=35)

plt.tight_layout()
fig_name = 'images/' + run_name + '/similarities.pdf'
plt.savefig(fig_name, dpi=300)

In [ ]:
# Scores distribution throughout optimization for the different AD
ordered_runs = ['smiles_validity_ecfp4', 'maxsim_ecfp4', 'maxsim_ecfp6', 'maxsim_ap', 'range_qed',
                'range_ecfp4', 'range_ecfp4_counts', 'range_physchem', 'range_physchem_range_ap', 'range_physchem_range_ecfp4', 'range_physchem_range_ecfp6',
                'range_physchem_range_ecfp4_counts', 'range_physchem_maxsim_ecfp4', 'range_physchem_maxsim_ecfp6', 
                'range_physchem_maxsim_ap']
fig, axs = plt.subplots(figsize = (20, 20), nrows=max(1, math.ceil(len(ordered_runs)/3)) , ncols=3, sharey=True)

fig.tight_layout(pad=1.5)

for i, run in enumerate(ordered_runs):
    ax = axs.flatten()[i]
    ax.set_title(run.replace('_', ' '))
    scores = np.array(scores_trajectories[runs.index(run)])
    median = np.median(scores, 1)
    q25 = np.percentile(scores, 5, axis=1)
    q75 = np.percentile(scores, 95, axis=1)
    ax.plot(median, c='b', label='Generated scores')
    ax.fill_between(
        np.arange(median.shape[0]), q25, q75, alpha=.1, color='b')
    ax.axhline(y=0.5, linewidth=1, ls='--', color='r', label='Predicted active threshold')
    ax.axhline(y=np.max(scores_test), linewidth=1, ls='--', color='g', label='Maximum test set score')
    ax.legend(loc=4)
fig.suptitle('Evolution of scores distributions', fontweight ="bold", y=1.05)
plt.tight_layout()
fig_name = 'images/' + run_name + '/scores_distribution.pdf'
plt.savefig(fig_name, dpi=300)

In [ ]:
internal_diversity = []
internal_diversity_sd = []
for i, run in enumerate(ordered_runs):
    all_curr_fps_generated = fps_generated_as_bitv[runs.index(run)]
    all_curr_internal_diversity = []
    nb_replicas = replica_per_run[run]
    for replica in range(nb_replicas):
        curr_fps_generated = all_curr_fps_generated[replica*top_kept_per_run[run][replica]:(replica+1)*top_kept_per_run[run][replica]]
        curr_internal_diversity = average_tanimoto_similarity(curr_fps_generated, curr_fps_generated)
        all_curr_internal_diversity.append(curr_internal_diversity)
    internal_diversity.append(np.mean(all_curr_internal_diversity))
    internal_diversity_sd.append(np.std(all_curr_internal_diversity))
    

In [ ]:
internal_diversity_display = [ str(round(1-m,2))+" ("+str(round(sd,2))+")" for m,sd in zip(internal_diversity,internal_diversity_sd)]

In [ ]:
novel_01_display = [ str(round(100*np.mean(n),1))+" ("+str(round(100*np.std(n),1))+")" for n in novel_01_all_per_rep]

In [ ]:
novel_01_display = [novel_01_display[runs.index(run)] for run in ordered_runs]

In [ ]:
novel_01_num = [ round(100*np.mean(n),1) for n in novel_01_all_per_rep]
novel_01_num = [novel_01_num[runs.index(run)] for run in ordered_runs]

In [ ]:
novel_recovered_display = [ str(round(100*np.mean(n),1))+" ("+str(round(100*np.std(n),1))+")" for n in novel_recovered_all_per_rep]

In [ ]:
novel_recovered_display= [novel_recovered_display[runs.index(run)] for run in ordered_runs]

In [ ]:
novel_recovered_num =  [ round(100*np.mean(n),1) for n in novel_recovered_all_per_rep]
novel_recovered_num = [novel_recovered_num[runs.index(run)] for run in ordered_runs]

In [ ]:
n_clusters_display = [n_clusters_display[runs.index(run)] for run in ordered_runs]

In [ ]:
entropies_display = [ str(round(np.mean(np.nan_to_num(n,nan=0)),2)) + " (" + str(round(np.std(np.nan_to_num(n,nan=0)),2)) + ")" for n in entropies_all_per_rep]
entropies_display = [entropies_display[runs.index(run)] for run in ordered_runs]

In [ ]:
ordered_runs = ['smiles_validity_ecfp4', 'maxsim_ecfp4', 'maxsim_ecfp6', 'maxsim_ap', 'range_qed',
                'range_ecfp4', 'range_ecfp4_counts', 'range_physchem', 'range_physchem_range_ap', 'range_physchem_range_ecfp4', 'range_physchem_range_ecfp6',
                'range_physchem_range_ecfp4_counts', 'range_physchem_maxsim_ecfp4', 'range_physchem_maxsim_ecfp6', 
                'range_physchem_maxsim_ap']
                
top_SAS = np.percentile(sas_dataset, 95)
sas_in_range = []
sas_in_range_sd = []
for i, run in enumerate(ordered_runs):
    all_curr_sas_in_range = []
    nb_replicas = replica_per_run[runs[runs.index(run)]]
    for replica in range(nb_replicas):
        curr_sas_in_range = SAS_for_results[i][replica*top_kept_per_run[run][replica]:(replica+1)*top_kept_per_run[run][replica]]
        curr_sas_in_range_x = [ x<top_SAS for x in curr_sas_in_range]
        all_curr_sas_in_range.append(curr_sas_in_range_x)
    sas_in_range.append(np.mean(all_curr_sas_in_range))
    sas_in_range_sd.append(np.std(all_curr_sas_in_range))

In [ ]:
sas_in_range_display = [ str(round(m*100,2))+" ("+str(round(sd*100,2))+")" for m,sd in zip(sas_in_range,sas_in_range_sd)]

In [ ]:
ordered_runs = ['smiles_validity_ecfp4', 'maxsim_ecfp4', 'maxsim_ecfp6', 'maxsim_ap', 'range_qed',
                'range_ecfp4', 'range_ecfp4_counts', 'range_physchem', 'range_physchem_range_ap', 'range_physchem_range_ecfp4', 'range_physchem_range_ecfp6',
                'range_physchem_range_ecfp4_counts', 'range_physchem_maxsim_ecfp4', 'range_physchem_maxsim_ecfp6', 
                'range_physchem_maxsim_ap']
                
bottom_QED = np.percentile(qed_dataset, 5)
qed_in_range = []
qed_in_range_sd = []
for i, run in enumerate(ordered_runs):
    all_curr_qed_in_range = []
    nb_replicas = replica_per_run[runs[runs.index(run)]]
    for replica in range(nb_replicas):
        #print(replica, top_kept_per_run[run][replica])
        curr_qed_in_range = QED_for_results[i][replica*top_kept_per_run[run][replica]:(replica+1)*top_kept_per_run[run][replica]]
        curr_qed_in_range = [ x>bottom_QED for x in curr_qed_in_range]
        all_curr_qed_in_range.append(curr_qed_in_range)
    qed_in_range.append(np.mean(all_curr_qed_in_range))
    qed_in_range_sd.append(np.std(all_curr_qed_in_range))

qed_in_range_display = [ str(round(m*100,2))+" ("+str(round(sd*100,2))+")" for m,sd in zip(qed_in_range,qed_in_range_sd)]

In [ ]:
cycle_size = []
cycle_size_sd = []
mw = []
mw_sd = []
heteroatoms = []
heteroatoms_sd = []
radicals = []
radicals_sd = []
for i, run in enumerate(ordered_runs):
    all_curr_smiles = smiles_list[runs.index(run)]
    all_curr_quantitative_analysis_results = []
    nb_replicas = replica_per_run[runs[runs.index(run)]]
    for replica in range(nb_replicas):
        #print(replica, top_kept_per_run[run][replica])
        curr_smiles = all_curr_smiles[replica*top_kept_per_run[run][replica]:(replica+1)*top_kept_per_run[run][replica]]
        curr_quantitative_analysis_results = quantitative_analysis(curr_smiles, smiles_test)
        all_curr_quantitative_analysis_results.append(curr_quantitative_analysis_results)
    cycle_size.append(np.mean([x[0] for x in all_curr_quantitative_analysis_results]))
    cycle_size_sd.append(np.std([x[0] for x in all_curr_quantitative_analysis_results]))
    mw.append(np.mean([x[1] for x in all_curr_quantitative_analysis_results]))
    mw_sd.append(np.std([x[1] for x in all_curr_quantitative_analysis_results]))
    heteroatoms.append(np.mean([x[2] for x in all_curr_quantitative_analysis_results]))
    heteroatoms_sd.append(np.std([x[2] for x in all_curr_quantitative_analysis_results]))
    radicals.append(np.mean([x[3] for x in all_curr_quantitative_analysis_results]))
    radicals_sd.append(np.std([x[3] for x in all_curr_quantitative_analysis_results]))

In [ ]:
cycle_size_display = [ str(round(m,1))+" ("+str(round(sd,1))+")" for m,sd in zip(cycle_size,cycle_size_sd)]
mw_display = [ str(round(m,1))+" ("+str(round(sd,1))+")" for m,sd in zip(mw,mw_sd)]
heteroatoms_display = [ str(round(m,1))+" ("+str(round(sd,1))+")" for m,sd in zip(heteroatoms,heteroatoms_sd)]
radicals_display = [ str(round(m,1))+" ("+str(round(sd,1))+")" for m,sd in zip(radicals,radicals_sd)]


In [ ]:
ordered_runs = ['smiles_validity_ecfp4', 'maxsim_ecfp4', 'maxsim_ecfp6', 'maxsim_ap', 'range_qed',
                'range_ecfp4', 'range_ecfp4_counts', 'range_physchem', 'range_physchem_range_ap', 'range_physchem_range_ecfp4', 'range_physchem_range_ecfp6',
                'range_physchem_range_ecfp4_counts', 'range_physchem_maxsim_ecfp4', 'range_physchem_maxsim_ecfp6', 
                'range_physchem_maxsim_ap']
df_w_sd = pd.DataFrame(list(zip(n_clusters_display, entropies_display, internal_diversity_display, novel_01_display, novel_recovered_display,
                           sas_in_range_display,qed_in_range_display, cycle_size_display, mw_display, heteroatoms_display, radicals_display)),
               columns =['Number of clusters', 'Entropies', 'Internal diversity','Actives recovered at distance <0.1', 
                         'Actives recovered', 'Valid SAS', 'Valid QED', 'Valid cycle sizes',
                        'Valid MW', 'Valid het-het bonds', 'Unpaired electrons'],
                 index = ordered_runs)

df_w_sd = df_w_sd.reset_index(drop=False)
df_w_sd['index']=[s.replace('_', ' ') for s in df_w_sd['index']]
df_w_sd.set_index('index',drop=True,inplace=True)

In [ ]:
df_w_sd

In [ ]:
# The dataframe with the global results without SD is used to generate the background colors of the table
ordered_runs = ['smiles_validity_ecfp4', 'maxsim_ecfp4', 'maxsim_ecfp6', 'maxsim_ap', 'range_qed',
                'range_ecfp4', 'range_ecfp4_counts', 'range_physchem', 'range_physchem_range_ap', 'range_physchem_range_ecfp4', 'range_physchem_range_ecfp6',
                'range_physchem_range_ecfp4_counts', 'range_physchem_maxsim_ecfp4', 'range_physchem_maxsim_ecfp6', 
                'range_physchem_maxsim_ap']
df = pd.DataFrame(list(zip(n_clusters, [x*100 for x in entropies], 100*(1 - np.array(internal_diversity)), [x for x in novel_01_num], [x for x in novel_recovered_num],
                           100 * np.array([np.mean(x<top_SAS) for x in SAS_for_results]),
                           100 * np.array([np.mean(x>bottom_QED) for x in QED_for_results]), cycle_size, mw, heteroatoms, radicals)),
               columns =['Number of clusters', 'Entropies', 'Internal diversity','Actives recovered at distance <0.1', 
                         'Actives recovered', 'Valid SAS', 'Valid QED', 'Valid cycle sizes',
                        'Valid MW', 'Valid het-het bonds', 'Unpaired electrons'],
                 index = ordered_runs)

df = df.reset_index(drop=False)
df['index']=[s.replace('_', ' ') for s in df['index']]
df.set_index('index',drop=True,inplace=True)

df_dummy = pd.DataFrame(list(zip([0, 10], [0, 1], [0, 100], [0, 100], [0, 100], [0, 100], [0, 100], [0, 100], [0, 100], [0, 100], [0, 100])),
               columns =['Number of clusters','Entropies', 'Internal diversity','Actives recovered at distance <0.1', 
                         'Actives recovered', 'Valid SAS', 'Valid QED', 'Valid cycle sizes',
                        'Valid MW', 'Valid het-het bonds', 'Unpaired electrons'],
                 index = ["dummy_low", "dummy_high"])
df = pd.concat([df, df_dummy])
s = df.style.background_gradient(cmap = 'BuGn', axis = None, subset=['Number of clusters', 'Entropies', 'Actives recovered at distance <0.1',
       'Actives recovered', 'Valid SAS', 'Valid QED', 'Internal diversity', 'Valid cycle sizes', 'Valid MW', 'Valid het-het bonds', 'Unpaired electrons'])


In [ ]:
df_4_display = pd.concat([df_w_sd, df_dummy])
s_4_display = df_4_display.style.background_gradient(cmap = 'BuGn', axis=None, gmap=df)
# Get latex code to display as table in article manuscript 
table_as_string = s_4_display.to_latex(convert_css=True)

In [ ]:
s_4_display

In [ ]:
out_filename = "results_" + run_name_low + "_raw.tex"
out_table_file = open(out_filename, "w")
n = out_table_file.write(table_as_string)
out_table_file.close()

# Qualitative analysis 

In [ ]:
runs

In [ ]:
# example with run "maxsim_ap"

# pick from runs list
ad = 'maxsim_ap'
show_grid_mols(ad)

In [ ]:
save_mols(ad, [0, 1, 4])

In [ ]:
save_mols(ad, [0, 1, 11])

In [ ]:
ad = 'smiles_validity_ecfp4'
show_grid_mols(ad)

In [ ]:
save_mols(ad, [0, 1, 24])

In [ ]:
ad = 'range_physchem_maxsim_ap'
show_grid_mols(ad)

In [ ]:
save_mols(ad, [4, 5, 49])

In [ ]:
ad = 'range_physchem_range_ap'
show_grid_mols(ad)

In [ ]:
save_mols(ad, [4, 20, 44])

In [ ]:
ad = 'range_ecfp4_counts'
show_grid_mols(ad)

In [ ]:
save_mols(ad, [0, 8, 49])

In [ ]:
ad = 'range_physchem_range_ecfp4_counts'
show_grid_mols(ad)

In [ ]:
save_mols(ad, [5, 6, 40])

In [ ]:
ad = 'range_physchem'
show_grid_mols(ad)

In [ ]:
save_mols(ad, [ 6, 8, 25, 29 ])

In [ ]:
ad = 'maxsim_ecfp4'
show_grid_mols(ad)

In [ ]:
save_mols(ad, [ 1, 30, 36 ])

In [ ]:
ad = 'range_physchem_maxsim_ecfp4'
show_grid_mols(ad)

In [ ]:
save_mols(ad, [ 2, 9, 29 ])

In [ ]:
ad = 'range_physchem_range_ecfp6'
show_grid_mols(ad)

In [ ]:
save_mols(ad, [ 4, 21, 28 ])

In [ ]:
ad = 'maxsim_ecfp6'
show_grid_mols(ad)

In [ ]:
save_mols(ad, [ 0, 17, 29 ])

In [ ]:
ad = 'range_physchem_range_ecfp4'
show_grid_mols(ad)

In [ ]:
save_mols(ad, [ 17, 19, 42 ])

In [ ]:
ad = 'range_qed'
show_grid_mols(ad)

In [ ]:
save_mols(ad, [ 2, 4, 19 ])

In [ ]:
ad = 'range_physchem_maxsim_ecfp6'
show_grid_mols(ad)

In [ ]:
save_mols(ad, [ 22, 31, 49 ])

In [ ]:
ad = 'range_ecfp4'
show_grid_mols(ad)

In [ ]:
save_mols(ad, [ 0, 9, 25 ])